In [32]:
pip install complexPyTorch

  Created wheel for complexPyTorch: filename=complexPyTorch-0.4-py3-none-any.whl size=7663 sha256=0fd9e0eda836d3577d431d5cd89b7cec72fd37fe764e3bb17aafd35b0ab1134b
  Stored in directory: c:\users\jeane\appdata\local\pip\cache\wheels\df\a5\f7\843d8ef45cc8317f4fc670427d4554f7646d4841fb08b8d362
Successfully built complexPyTorch
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#https://github.com/wavefrontshaping/complexPyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexPyTorch.complexLayers import ComplexBatchNorm2d, ComplexConv2d, ComplexLinear
from complexPyTorch.complexFunctions import complex_relu, complex_max_pool2d


In [5]:
import numpy as np
import random 
from collections import namedtuple, deque 

##Importing the model (function approximator for Q-table)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## équivalent à baselines/ddqn.py

In [ ]:
##image_width représente le nombre d'action qu'on a
##nos masques doivent être de la forme [batch_size, 1, max_width = 28], 
##nos kspace au niveau du ddqn seront de la forme [batch_size, coil = 1, max_width = 28, max_width,] ==>> on a + ib
##au niveau de spirit on passera à  [batch_size, coil = 1, max_width = 28, max_width, 2] ##forme de départ

In [ ]:
##prendre un dico d'observation (recons , mask), et le fournit en input au qnetwork
##
def _encode_obs_myown(kspace_reconst, mask, img_reconst) :
    
    #transform into [batch_size, n_coils, height, width, 2]
    kspace_reconst = torch.cat( (torch.unsqueeze(kspace_reconst.real, -1),torch.unsqueeze(kspace_reconst.imag, -1) ),-1) #shape [n, coil = 1, max_width = 28, max_width, 2]
    
    return {"kspace_reconst", "mask", "img_reconst"}


def _decode_obs_dict_myown(obs: Dict[str, Any]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    
   ## obs["kspace_reconst"] a la forme [batch_size, num_channels, img_height, img_width, 2]
    kspace_reconst = obs["kspace_reconst"][...,0] +1j*obs["kspace_reconst"][...,1]
    
    mask = obs["mask"]
    img_reconst = obs["img_reconst"]
    bs = obs_tensor.shape[0] #batch_size

    mask = obs_tensor[:, 0, -1, :] ##ceci reviendra dont à 
    mask = mask.contiguous().view(bs, 1, 1, -1) #(batch_size, 1, 1, img_width)

    return kspace_reconst, mask, img_reconst

In [ ]:
## ce q network va prendre en entrée des kspace à valeurs complexes et faire sortir les q-values des actions

##l'architecture du modèle utilié vient de : https://github.com/wavefrontshaping/complexPyTorch

class q_ComplexNet(nn.Module):
    
    def __init__(self, opts, ):
        
        super(q_ComplexNet, self).__init__()
        self.conv1 = ComplexConv2d(1, 10, 5, 1)
        self.bn  = ComplexBatchNorm2d(10)
        self.conv2 = ComplexConv2d(10, 20, 5, 1)
        self.fc1 = ComplexLinear(4*4*20, 500)
        self.fc2 = ComplexLinear(500, opts.image_width)
             
    def forward(self, obs: Dict[str, Any]) -> torch.Tensor:
        
        """Predicts action values.
        Args:
            obs(torch.Tensor): The observation tensor. Once decoded, it only uses the mask
                               information. If ``__init__(..., ignore_mask=True)``, it will
                               additionally use the mask only to deduce the time step.
        Returns:
            torch.Tensor: Q-values for all actions at the given observation.
        Note:
            Values corresponding to active k-space columns in the observation are manually
            set to ``1e-10``.
        """
        
        kspace_reconst, mask, _ = _decode_obs_dict_myown(obs ## il faudrait faire en sorte que l'état st (observation) soit un dico compotant le kspace reconstruit et le mask
        previous_actions = mask.squeeze() ##??? [batch_size, width = 28 ]
        
        input_tensor = kspace_reconst ## ils ont  mis mask dans le cas data-specific

        
        x = self.conv1(input_tensor)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = self.bn(x)
        x = self.conv2(x)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = x.view(-1,4*4*20)
        x = self.fc1(x)
        x = complex_relu(x)
        x = self.fc2(x)
        
        value = x.abs()  ### est ce que ça a de sens ???? Pour le moment, mon objectif est d'avoir un modèle qui a une démarche de RL, après on peut enlever ce qui ne marche pas.
        

        return value - 1e10 * previous_actions 

"""a = torch.randn((4,1,28,28,2))

z = a[...,0]+1j*b[...,1]

dd = q_ComplexNet()
so = dd(z)

so.shape"""    

In [ ]:
def _get_model(options):
    if options.dqn_model_type == "q_ComplexNet":
        return q_ComplexNet(options)
    
    else:
        raise ValueError("Unknown model specified for DDQN.")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

###remplacer posiblement options par un argparse, cet l'équivalent d'un config
options = {"BUFFER_SIZE": int(1e5),  #replay buffer size
          "BATCH_SIZE": 64,   # minibatch size  
          "GAMMA": 0.5,       # discount factor
          "TAU":1e-3,         # for soft update of target parameters
           "LR":5e-4,         # learning rate
          "UPDATE_EVERY":4,   # how often to update the network
          
          "dqn_model_type": "q_ComplexNet",
          "budget":5, ##je veux prendre 2 colonnes au départ et le modèle va déterminer les 5 restants ; on  a au total 7 colonnes, soit une accélération de 4
          "image_width":28, ## it is the action_size
          "seed" : 123}


class DDQN():
    """Interacts with and learns form environment."""
    
    def __init__(self, options):
        """Initialize an Agent object.
        
        Params
        =======
            options
        """        

        self.options = options
        self.seed = random.seed(options.seed)
        
        
        
        #Q- Network
        self.qnetwork_local = _get_model(options).to(device)
        self.qnetwork_target = _get_model(options).to(device)
        
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr = options.LR)
        
        # Replay memory 
        self.memory = ReplayBuffer(self.options.image_width, self.options.BUFFER_SIZE, self.options.BATCH_SIZE, self.options.seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        
        self.t_step = 0
        
        
        
    def step(self, state, action, reward, next_step, done): ##state here is observation  dict, next_step is the same thing
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_step, done)

        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step+1)% self.options.UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn

            if len(self.memory)>self.options.BATCH_SIZE:
                experience = self.memory.sample()
                self.learn(experience, self.options.GAMMA)
                
                
                
    def act(self, state, eps = 0):
        """Returns action for given state as per current policy
        Params
        =======
            state : kspace_reconst, mask, img_reconst
            eps (float): epsilon, for epsilon-greedy action selection
        """
                
        self.qnetwork_local.eval()
        with torch.no_grad():
            
            action_values = self.qnetwork_local(state)
            
        self.qnetwork_local.train()

        #Epsilon -greedy action selction ; en prenant l'action, il faut modifier le masque en conséquence
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
            
            
    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        =======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples ; with s an observation dico
            gamma (float): discount factor
        """
        states, actions, rewards, next_state, dones = experiences
        ## TODO: compute and minimize the loss
        criterion = torch.nn.SmoothL1Loss()
        self.qnetwork_local.train()
        self.qnetwork_target.eval()
        
        #shape of output from the model (batch_size,action_dim) = (batch_size,28)
        predicted_targets = self.qnetwork_local(states).gather(1,actions)

        #################Updates for Double DQN learning###########################
        self.qnetwork_local.eval()
        with torch.no_grad():
            actions_q_local = self.qnetwork_local(next_states).detach().max(1)[1].unsqueeze(1).long()
            labels_next = self.qnetwork_target(next_states).gather(1,actions_q_local)
        self.qnetwork_local.train()
        ############################################################################

        # .detach() ->  Returns a new Tensor, detached from the current graph.
        labels = rewards + (gamma* labels_next*(1-dones))

        loss = criterion(predicted_targets,labels).to(device)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local,self.qnetwork_target,self.options.TAU)
            
            
            
            
            
    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        =======
            local model (PyTorch model): weights will be copied from
            target model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter
        """
        for target_param, local_param in zip(target_model.parameters(),
                                           local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1-tau)*target_param.data)
            
            
class ReplayBuffer:
    """Fixed -size buffer to store experience tuples."""
    
    def __init__(self, options, seed):
        """Initialize a ReplayBuffer object.
        
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.options = options
        self.action_size = self.options.image_width
        self.memory = deque(maxlen=self.options.BUFFER_SIZE)
        self.batch_size = self.options.BATCH_SIZE
        self.experiences = namedtuple("Experience", field_names=["state",
                                                               "action",
                                                               "reward",
                                                               "next_state",
                                                               "done"])
        self.seed = random.seed(seed)
        
        
    def add(self,state, action, reward, next_state,done):
        """Add a new experience to memory."""
        e = self.experiences(state,action,reward,next_state,done)
        self.memory.append(e)
        
        
    def concat_dict_element(dict_sample,dim=0) : 
    """concat the tensor that are in different state dictionnary or next_state"""
    dico = {}
    for key in dict_sample[0].keys():
        dico[key] = torch.cat( [ dic[key] for dic in dict_sample ], 0).to(device) #.float .complex ?
     
    return dico ##this dico will contain a batch of kspace reconstruction, a batch of mask and a batch of image reconstruction
    
        
    def sample(self):
        """Randomly sample a batch of experiences from memory"""
        experiences = random.sample(self.memory,k=self.batch_size)
        
        ##take an experience and extract the keys that are in the state dictionnary
        
        
        states = concat_dict_element([e.state for e experiences if e is not None],dim=0)
        
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        
        next_states = concat_dict_element([e.next_state for e experiences if e is not None],dim=0)
        
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states,actions,rewards,next_states,dones)
    
    
    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [ ]:

def dqn(n_episodes= 200, max_t = 1000, eps_start=1.0, eps_end = 0.01,
       eps_decay=0.996):
    """Deep Q-Learning
    
    Params
    ======
        n_episodes (int): maximum number of training epsiodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon 
        eps_decay (float): mutiplicative factor (per episode) for decreasing epsilon
        
    """
    scores = [] # list containing score from each episode
    scores_window = deque(maxlen=100) # last 100 scores
    eps = eps_start
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state,eps)
            next_state,reward,done,_ = env.step(action)
            agent.step(state,action,reward,next_state,done)
            ## above step decides whether we will train(learn) the network
            ## actor (local_qnetwork) or we will fill the replay buffer
            ## if len replay buffer is equal to the batch size then we will
            ## train the network or otherwise we will add experience tuple in our 
            ## replay buffer.
            state = next_state
            score += reward
            if done:
                break
            scores_window.append(score) ## save the most recent score
            scores.append(score) ## sae the most recent score
            eps = max(eps*eps_decay,eps_end)## decrease the epsilon
            print('\rEpisode {}\tAverage Score {:.2f}'.format(i_episode,np.mean(scores_window)), end="")
            if i_episode %100==0:
                print('\rEpisode {}\tAverage Score {:.2f}'.format(i_episode,np.mean(scores_window)))
                
            if np.mean(scores_window)>=200.0:
                print('\nEnvironment solve in {:d} epsiodes!\tAverage score: {:.2f}'.format(i_episode-100,
                                                                                           np.mean(scores_window)))
                torch.save(agent.qnetwork_local.state_dict(),'checkpoint.pth')
                break
    return scores


In [ ]:
class DDQNTrainer:
    """DDQN Trainer for active MRI acquisition.
    Configuration for the trainer is provided by argument ``options``. Must contain the
    following fields:
        - "checkpoints_dir"(str): The directory where the model will be saved to (or
          loaded from).
        - dqn_batch_size(int): The batch size to use for updates.
        - dqn_burn_in(int): How many steps to do before starting updating parameters.
        - dqn_normalize(bool): ``True`` if running mean/st. deviation should be maintained
          for observations.
        - dqn_only_test(bool): ``True`` if the model will not be trained, thus only will
          attempt to read from checkpoint and load only weights of the network (ignoring
          training related information).
        - dqn_test_episode_freq(optional(int)): How frequently (in number of env steps)
          to perform test episodes.
        - freq_dqn_checkpoint_save(int): How often (in episodes) to save the model.
        - num_train_steps(int): How many environment steps to train for.
        - replay_buffer_size(int): The capacity of the replay buffer.
        - resume(bool): If true, will try to load weights from the checkpoints dir.
        - num_test_episodes(int): How many test episodes to periodically evaluate for.
        - seed(int): Sets the seed for the environment when running evaluation episodes.
        - reward_metric(str): Which of the ``env.scores_keys()`` is used as reward. Mainly
          used for logging purposes.
        - target_net_update_freq(int): How often (in env's steps) to update the target
          network.
    Args:
        options(``argparse.Namespace``): Options for the trainer.
        env(``activemri.envs.ActiveMRIEnv``): Env for which the policy is trained.
        device(``torch.device``): Device to use.
    """

    def __init__(
        self,
        options: argparse.Namespace,
        env: mri_envs.ActiveMRIEnv,
        device: torch.device,
    ):
        self.options = options
        self.env = env
        self.options.image_width = self.env.kspace_width
        self.steps = 0
        self.episode = 0
        self.best_test_score = -np.inf
        self.device = device
        self.replay_memory = None

        self.window_size = 1000
        self.reward_images_in_window = np.zeros(self.window_size)
        self.current_score_auc_window = np.zeros(self.window_size)



        print("Creating DDQN model.")

        self.logger.info(
            f"Creating replay buffer with capacity {options.mem_capacity}."
        )

        # ------- Create replay buffer and networks ------
        # See _encode_obs_dict() for tensor format
        self.obs_shape = (2, self.env.kspace_height + 2, self.env.kspace_width)
        self.replay_memory = replay_buffer.ReplayMemory(
            options.mem_capacity,
            self.obs_shape,
            self.options.dqn_batch_size,
            self.options.dqn_burn_in,
            
        )
        print("Created replay buffer.")
        
        self.policy = DDQN(device, self.replay_memory, self.options)
        self.target_net = DDQN(device, None, self.options)
        self.target_net.eval()
        self.logger.info(
            f"Created neural networks with {self.env.action_space.n} outputs."
        )

        # ------- Files used to communicate with DDQNTester ------
        self.folder_lock_path = DDQNTrainer.get_lock_filename(
            self.options.checkpoints_dir
        )
        with _get_folder_lock(self.folder_lock_path):
            # Write options so that tester can read them
            with open(
                DDQNTrainer.get_options_filename(self.options.checkpoints_dir), "wb"
            ) as f:
                pickle.dump(self.options, f)
            # Remove previous done file since training will start over
            done_file = DDQNTrainer.get_done_filename(self.options.checkpoints_dir)
            if os.path.isfile(done_file):
                os.remove(done_file)

    @staticmethod
    def get_done_filename(path):
        return os.path.join(path, "DONE")

    @staticmethod
    def get_name_latest_checkpoint(path):
        return os.path.join(path, "policy_checkpoint.pth")

    @staticmethod
    def get_options_filename(path):
        return os.path.join(path, "options.pickle")

    def _max_replay_buffer_size(self):
        return min(self.options.num_train_steps, self.options.replay_buffer_size)

    def load_checkpoint_if_needed(self):
        if self.options.dqn_only_test or self.options.resume:
            policy_path = os.path.join(self.options.dqn_weights_path)
            if os.path.isfile(policy_path):
                self.load(policy_path)
                self.logger.info(f"Loaded DQN policy found at {policy_path}.")
            else:
                self.logger.warning(f"No DQN policy found at {policy_path}.")
                if self.options.dqn_only_test:
                    raise FileNotFoundError

    def _train_dqn_policy(self):
        """ Trains the DQN policy. """
        print(
            f"Starting training at step {self.steps}/{self.options.num_train_steps}. "
            f"Best score so far is {self.best_test_score}."
        )

        steps_epsilon = self.steps
        while self.steps < self.options.num_train_steps:
            self.logger.info("Episode {}".format(self.episode + 1))

            # Evaluate the current policy
            if self.options.dqn_test_episode_freq and (
                self.episode % self.options.dqn_test_episode_freq == 0
            ):
                
                test_scores, _ = evaluation.evaluate(
                    self.env,
                    self.policy,
                    self.options.num_test_episodes,
                    self.options.seed,
                    "val",
                )
                
                self.env.set_training()
                auc_score = test_scores[self.options.reward_metric].sum(axis=1).mean()
                if "mse" in self.options.reward_metric:
                    auc_score *= -1
                if auc_score > self.best_test_score:
                    policy_path = os.path.join(
                        self.options.checkpoints_dir, "policy_best.pt"
                    )
                    self.save(policy_path)
                    self.best_test_score = auc_score
                    self.logger.info(
                        f"Saved DQN model with score {self.best_test_score} to {policy_path}."
                    )

            # Save model periodically
            if self.episode % self.options.freq_dqn_checkpoint_save == 0:
                self.checkpoint(save_memory=False)

            # Run an episode and update model
            obs, meta = self.env.reset()
            msg = ", ".join(
                [
                    f"({meta['fname'][i]}, {meta['slice_id'][i]})"
                    for i in range(len(meta["slice_id"]))
                ]
            )
            self.logger.info(f"Episode started with images {msg}.")
            all_done = False
            total_reward = 0
            auc_score = 0
            while not all_done:
                epsilon = _get_epsilon(steps_epsilon, self.options)
                action = self.policy.get_action(obs, eps_threshold=epsilon)
                next_obs, reward, done, meta = self.env.step(action)
                auc_score += meta["current_score"][self.options.reward_metric]
                all_done = all(done)
                self.steps += 1
                obs_tensor = _encode_obs_dict(obs)
                next_obs_tensor = _encode_obs_dict(next_obs)
                batch_size = len(obs_tensor)
                for i in range(batch_size):
                    self.policy.add_experience(
                        obs_tensor[i], action[i], next_obs_tensor[i], reward[i], done[i]
                    )

                update_results = self.policy.update_parameters(self.target_net)
                torch.cuda.empty_cache()
                if self.steps % self.options.target_net_update_freq == 0:
                    self.logger.info("Updating target network.")
                    self.target_net.load_state_dict(self.policy.state_dict())
                steps_epsilon += 1

                # Adding per-step tensorboard logs
                if self.steps % 250 == 0:
                    self.logger.debug("Writing to tensorboard.")
                    self.writer.add_scalar("epsilon", epsilon, self.steps)
                    if update_results is not None:
                        self.writer.add_scalar(
                            "loss", update_results["loss"], self.steps
                        )
                        self.writer.add_scalar(
                            "grad_norm", update_results["grad_norm"], self.steps
                        )
                        self.writer.add_scalar(
                            "mean_q_value", update_results["q_values_mean"], self.steps
                        )
                        self.writer.add_scalar(
                            "std_q_value", update_results["q_values_std"], self.steps
                        )

                total_reward += reward
                obs = next_obs

            # Adding per-episode tensorboard logs
            total_reward = total_reward.mean().item()
            auc_score = auc_score.mean().item()
            self.reward_images_in_window[self.episode % self.window_size] = total_reward
            self.current_score_auc_window[self.episode % self.window_size] = auc_score
            self.writer.add_scalar("episode_reward", total_reward, self.episode)
            self.writer.add_scalar(
                "average_reward_images_in_window",
                np.sum(self.reward_images_in_window)
                / min(self.episode + 1, self.window_size),
                self.episode,
            )
            self.writer.add_scalar(
                "average_auc_score_in_window",
                np.sum(self.current_score_auc_window)
                / min(self.episode + 1, self.window_size),
                self.episode,
            )

            self.episode += 1

        self.checkpoint()

        # Writing DONE file with best test score
        with _get_folder_lock(self.folder_lock_path):
            with open(
                DDQNTrainer.get_done_filename(self.options.checkpoints_dir), "w"
            ) as f:
                f.write(str(self.best_test_score))

        return self.best_test_score

    def __call__(self):
        self.load_checkpoint_if_needed()
        return self._train_dqn_policy()

    def checkpoint(self, save_memory=True):
        policy_path = DDQNTrainer.get_name_latest_checkpoint(
            self.options.checkpoints_dir
        )
        self.save(policy_path)
        self.logger.info(f"Saved DQN checkpoint to {policy_path}")
        if save_memory:
            self.logger.info("Now saving replay memory.")
            memory_path = self.replay_memory.save(
                self.options.checkpoints_dir, "replay_buffer.pt"
            )
            self.logger.info(f"Saved replay buffer to {memory_path}.")

    def save(self, path):
        with _get_folder_lock(self.folder_lock_path):
            torch.save(
                {
                    "dqn_weights": self.policy.state_dict(),
                    "target_weights": self.target_net.state_dict(),
                    "options": self.options,
                    "episode": self.episode,
                    "steps": self.steps,
                    "best_test_score": self.best_test_score,
                    "reward_images_in_window": self.reward_images_in_window,
                    "current_score_auc_window": self.current_score_auc_window,
                },
                path,
            )

    def load(self, path):
        checkpoint = torch.load(path)
        self.policy.load_state_dict(checkpoint["dqn_weights"])
        self.episode = checkpoint["episode"] + 1
        if not self.options.dqn_only_test:
            self.target_net.load_state_dict(checkpoint["target_weights"])
            self.steps = checkpoint["steps"]
            self.best_test_score = checkpoint["best_test_score"]
            self.reward_images_in_window = checkpoint["reward_images_in_window"]
            self.current_score_auc_window = checkpoint["current_score_auc_window"]

In [ ]:
agent = Agent(state_size=8,action_size=4,seed=0)

scores= dqn()

#plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)),scores)
plt.ylabel('Score')
plt.xlabel('Epsiode #')
plt.show()

In [89]:
a = torch.randn((2,1,5,5))
a

tensor([[[[ 0.3956,  0.1844,  0.3374, -0.5964,  0.1253],
          [-0.4805,  1.1258,  0.2264,  1.2968, -2.7566],
          [ 0.6585, -1.4463, -0.5388,  0.6482,  0.0750],
          [-0.7434, -0.1443,  0.2038,  0.7724,  0.7934],
          [-1.4173, -1.3582,  0.1742,  0.4491,  0.7307]]],


        [[[ 0.8428, -1.7390,  1.0562,  2.1782, -1.5747],
          [-0.2892, -1.9416,  0.0620, -0.3615,  0.4342],
          [ 0.1588, -0.1162, -1.3501,  0.9667,  2.2290],
          [ 0.7906, -0.4644, -0.0308,  0.3798, -1.2911],
          [-0.7894,  0.8741,  2.8280,  0.1516, -1.6244]]]])

In [95]:
b = torch.from_numpy(np.expand_dims(np.eye(5), (0,1)))
b

tensor([[[[1., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 1.]]]], dtype=torch.float64)

In [100]:
c = np.expand_dims(np.fliplr(np.eye(5)) , (0,1))
c = torch.from_numpy(c.copy())
c

tensor([[[[0., 0., 0., 0., 1.],
          [0., 0., 0., 1., 0.],
          [0., 0., 1., 0., 0.],
          [0., 1., 0., 0., 0.],
          [1., 0., 0., 0., 0.]]]], dtype=torch.float64)

In [102]:
d = torch.cat((b,c),0)
d

tensor([[[[1., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.],
          [0., 0., 0., 0., 1.]]],


        [[[0., 0., 0., 0., 1.],
          [0., 0., 0., 1., 0.],
          [0., 0., 1., 0., 0.],
          [0., 1., 0., 0., 0.],
          [1., 0., 0., 0., 0.]]]], dtype=torch.float64)

In [104]:
e = d*1e7
e 

tensor([[[[10000000.,        0.,        0.,        0.,        0.],
          [       0., 10000000.,        0.,        0.,        0.],
          [       0.,        0., 10000000.,        0.,        0.],
          [       0.,        0.,        0., 10000000.,        0.],
          [       0.,        0.,        0.,        0., 10000000.]]],


        [[[       0.,        0.,        0.,        0., 10000000.],
          [       0.,        0.,        0., 10000000.,        0.],
          [       0.,        0., 10000000.,        0.,        0.],
          [       0., 10000000.,        0.,        0.,        0.],
          [10000000.,        0.,        0.,        0.,        0.]]]],
       dtype=torch.float64)

In [113]:
a_bis = torch.where(d==1, torch.tensor(1e7, dtype =a.dtype), a)
a_bis

tensor([[[[ 1.0000e+07,  1.8441e-01,  3.3739e-01, -5.9643e-01,  1.2526e-01],
          [-4.8053e-01,  1.0000e+07,  2.2641e-01,  1.2968e+00, -2.7566e+00],
          [ 6.5847e-01, -1.4463e+00,  1.0000e+07,  6.4818e-01,  7.5019e-02],
          [-7.4336e-01, -1.4428e-01,  2.0379e-01,  1.0000e+07,  7.9336e-01],
          [-1.4173e+00, -1.3582e+00,  1.7416e-01,  4.4911e-01,  1.0000e+07]]],


        [[[ 8.4277e-01, -1.7390e+00,  1.0562e+00,  2.1782e+00,  1.0000e+07],
          [-2.8924e-01, -1.9416e+00,  6.2042e-02,  1.0000e+07,  4.3420e-01],
          [ 1.5880e-01, -1.1625e-01,  1.0000e+07,  9.6666e-01,  2.2290e+00],
          [ 7.9059e-01,  1.0000e+07, -3.0757e-02,  3.7985e-01, -1.2911e+00],
          [ 1.0000e+07,  8.7413e-01,  2.8280e+00,  1.5158e-01, -1.6244e+00]]]])

In [109]:
a.dtype

torch.float32

In [111]:
torch.tensor(1, dtype =a.dtype)

tensor(1.)

In [115]:
a=2
a *=4
a

8

In [1]:
a = False

if  not a:
    print("yes")

yes


In [10]:
from easydict import EasyDict as edict

d = edict({})
d[str(1)] = 2
d

{'1': 2}

In [11]:
d.r =3

In [12]:
d

{'1': 2, 'r': 3}

In [8]:
d = {}

d[1] = "r"